In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_object_dtype

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

root_dir = os.path.abspath('..')
sys.path.append(os.path.join(root_dir, 'src/utils'))

In [2]:
DATA_FOLDER = os.path.join(root_dir, 'data/')
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'
SEGMENTS_FILE = 'Segments.xlsx'

In [3]:
def calc_nan_percent(df):
    return round(100 * df.isna().sum() / df.shape[0], 2)

# Загрузка данных

In [4]:
segments = pd.read_excel(os.path.join(DATA_FOLDER, SEGMENTS_FILE))
display(segments)

,Номер сегмента,Возраст,Пол,Интересы
0,1,"25-34,35-41",Ж,-
1,2,"25-34,35-42",М,Пиво
2,3,"25-34,35-43",Ж,Дети
3,4,"18-24,25-34,35-44","М, Ж",Животные
4,5,"18-24,25-34,35-45","М, Ж",-


In [5]:
train = pd.read_csv(os.path.join(DATA_FOLDER, TRAIN_FILE))
display(train)

,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,Забайкальский Край,Чита,android,10.0
1,4,NaN,NaN,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,Оренбургская область,Оренбург,Android,10.0.0
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,Санкт-Петербург,Санкт-Петербург,android,9.0
3,5,NaN,NaN,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,Свердловская область,Екатеринбург,android,9
4,4,NaN,NaN,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,Московская область,Звенигород,android,6.0.1
...,...,...,...,...,...,...,...,...,...,...
44854511,3,Games,Simulation,1068204657,2021-08-07 17:19:23,MSK,Краснодарский край,Краснодар,ios,14.4.2
44854512,3,Games,Puzzle,com.easybrain.nonogram.color,2021-08-02 09:17:16,MSK,Владимирская область,Владимир,android,11.0
44854513,5,Games,Arcade,com.nordcurrent.canteenhd,2021-09-16 09:26:38,MSK,Брянская область,Брянск,android,5.1
44854514,4,NaN,NaN,com.fugo.wow,2021-07-09 18:02:33,MSK,Татарстан,Казань,Android,7.1.2


In [6]:
test = pd.read_csv(os.path.join(DATA_FOLDER, TEST_FILE))
display(test)

,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10.0
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10.0
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14.4.0
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9.0.0
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10.0
...,...,...,...,...,...,...,...,...,...
11213624,NaN,NaN,com.phonemaster.jewelhunter,2021-08-04 05:55:26,MSK,Санкт-Петербург,Санкт-Петербург,android,8.1.0
11213625,NaN,NaN,com.linkdesks.jewellegend,2021-09-17 21:45:02,MSK,Ставрополье,Пятигорск,android,10
11213626,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 10:24:31,MSK+6,Забайкальский Край,Чита,android,11.0
11213627,NaN,NaN,com.onebutton.mrsuper,2021-07-05 16:02:47,MSK+2,Башкортостан,Белорецк,android,6.0.1


# Осмотр данных

In [7]:
display(train.info())
display(calc_nan_percent(train))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44854516 entries, 0 to 44854515
Data columns (total 10 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   Segment          int64 
 1   gamecategory     object
 2   subgamecategory  object
 3   bundle           object
 4   created          object
 5   shift            object
 6   oblast           object
 7   city             object
 8   os               object
 9   osv              object
dtypes: int64(1), object(9)
memory usage: 3.3+ GB


None

Segment             0.00
gamecategory       37.81
subgamecategory    37.83
bundle              0.04
created             0.00
shift               8.01
oblast              7.70
city               10.70
os                  0.00
osv                 0.00
dtype: float64

In [8]:
display(test.info())
display(calc_nan_percent(test))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11213629 entries, 0 to 11213628
Data columns (total 9 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   gamecategory     object
 1   subgamecategory  object
 2   bundle           object
 3   created          object
 4   shift            object
 5   oblast           object
 6   city             object
 7   os               object
 8   osv              object
dtypes: object(9)
memory usage: 770.0+ MB


None

gamecategory       37.82
subgamecategory    37.83
bundle              0.04
created             0.00
shift               8.00
oblast              7.70
city               10.71
os                  0.00
osv                 0.00
dtype: float64

In [9]:
display(train['shift'].unique())
display(test['shift'].unique())

array(['MSK+6', 'MSK+2', 'MSK', 'MSK+3', nan, 'MSK+1', 'MSK+4', 'MSK-1',
       'MSK+5', 'MSK+7', 'MSK+9', 'MSK+8'], dtype=object)

array(['MSK+2', 'MSK', 'MSK+1', 'MSK+6', 'MSK+3', 'MSK+4', 'MSK-1', nan,
       'MSK+5', 'MSK+7', 'MSK+9', 'MSK+8'], dtype=object)

In [10]:
cities_train = train['city'].str.upper().unique()
cities_test = test['city'].str.upper().unique()
print(len(cities_train))
print(len(cities_test))

2499
2466


In [11]:
print(100 * train[train['shift'].isnull() & train['city'].notna()].shape[0] / train['shift'].isnull().sum())

3.783467691408045


In [12]:
print(100 * train[train['shift'].isnull() & train['oblast'].notna()].shape[0] / train['shift'].isnull().sum())

3.7835233838742464


In [13]:
print(100 * train[train['city'].isnull() & train['oblast'].notna()].shape[0] / train['city'].isnull().sum())

28.014921691536152


In [14]:
print(100 * train[train['oblast'].isnull() & train['city'].notna()].shape[0] / train['oblast'].isnull().sum())

0.0


In [15]:
100 * train['city'].str.upper().value_counts() / train['city'].str.upper().value_counts().sum()

МОСКВА             19.085050
САНКТ-ПЕТЕРБУРГ     7.593280
ЕКАТЕРИНБУРГ        6.722314
КРАСНОДАР           5.728177
КАЗАНЬ              4.721577
                     ...    
ОСОВЦЫ              0.000002
ДУБОВИКИ            0.000002
ЛОГОЙСК             0.000002
НОВОГРУДОК          0.000002
УРДОМА              0.000002
Name: city, Length: 2498, dtype: float64

In [16]:
100* test['city'].str.upper().value_counts() / test['city'].str.upper().value_counts().sum()

МОСКВА             19.090384
САНКТ-ПЕТЕРБУРГ     7.590310
ЕКАТЕРИНБУРГ        6.730807
КРАСНОДАР           5.721603
КАЗАНЬ              4.720040
                     ...    
ИЗБОРСК             0.000010
ЛОКТИОНОВО          0.000010
АГАПОВКА            0.000010
ПЕТРЯЕВКА           0.000010
ОНОХИНО             0.000010
Name: city, Length: 2465, dtype: float64

# Функция

In [17]:
holidays_list = [
    '2021-12-31',
    '2021-01-01',
    '2021-02-23',
    '2021-03-08'
]

holidays = set()


In [63]:
def create_time_features(df,
                        dt_target='loc',
                        datetime_col_msk='created',
                        shift_col = 'shift',
                        fill_shift_na=True,
                        shift_filler='MSK',
                        dt_format = '%Y-%m-%d %H:%M:%S',
                        tags=False
                        ):

    df = df.copy()

    prefix = 'msk_'
    shift = df[shift_col].copy()

    if dt_target == 'loc':
        prefix = 'loc_'
        if fill_shift_na:
            shift = shift.fillna(shift_filler)
        else:
            shift = shift.dropna()

    datetime_msk = pd.to_datetime(df.loc[shift.index, datetime_col_msk], format=dt_format)
    datetime_loc = datetime_msk.copy()

    if dt_target == 'loc':
        shift = shift.str.replace('MSK', '0')
        sign = shift.str.replace(r'[^+-]', '', regex=True)
        is_forward = sign == '+'
        shift_value = pd.to_timedelta(shift.str.replace(r'[^0-9]', '', regex=True).astype(int), 'H')
        datetime_loc[is_forward] += shift_value[is_forward]
        datetime_loc[~is_forward] -= shift_value[~is_forward]

    result = pd.DataFrame(dtype=object, index=df.index)

    minute = datetime_loc.dt.minute
    hour = datetime_loc.dt.hour
    weekday = datetime_loc.dt.weekday
    day = datetime_loc.dt.day
    month = datetime_loc.dt.month
    year = datetime_loc.dt.year

    academic_year = ((month <=5) | (month >=9)).astype(int)
    first_september = ((month == 9) & (day == 1)).astype(int)
    week_before_first_september = ((month == 8) & (day >= 25)).astype(int)

    early_morning = \
        ((hour >= 5) & (hour < 8)).astype(int)

    morning = \
        ((hour >= 8) & (hour < 11)).astype(int)

    day = \
        ((hour >= 11) & (hour < 18)).astype(int)
    
    evening = \
        ((hour >= 18) & (hour < 21)).astype(int)
    
    late_evening = \
        ((hour >= 21) & (hour <= 23)).astype(int)
    
    night = \
        ((hour >= 0) & (hour < 5)).astype(int)


    result[f'{prefix}minute'] = minute
    result[f'{prefix}hour'] = hour
    result[f'{prefix}day'] = day
    result[f'{prefix}month'] = month
    result[f'{prefix}year'] = year
    result[f'{prefix}weekday'] = weekday

    result[f'{prefix}is_weekend'] = (weekday.isin([5, 6])).astype(int)

    result[f'{prefix}days_to_weekend'] = 5 - result[f'{prefix}weekday']
    result[f'{prefix}days_to_weekend'].replace({-1:0}, inplace=True)

    
    result[f'{prefix}is_academic_year'] = academic_year
    result[f'{prefix}is_first_september'] = first_september
    result[f'{prefix}is_week_before_first_september'] = week_before_first_september

    result[f'{prefix}is_early_morning'] = early_morning
    result[f'{prefix}is_morning'] = morning
    result[f'{prefix}is_day'] = day
    result[f'{prefix}is_evening'] = evening
    result[f'{prefix}is_late_evening'] = late_evening
    result[f'{prefix}is_night'] = night

    bad_year = result[f'{prefix}year']==1970

    result.drop(columns=[f'{prefix}year'], inplace=True)

    for col in result.columns:
        result.loc[bad_year, col] = np.nan

    return result

In [92]:
def get_tags_from_time_features(df, tags_cols=None, tags_dict=None, delimiter=';'):

    if 'loc_day' in df.columns:
        prefix = 'loc_'
    elif 'msc_day' in df.columns:
        prefix = 'msc_'
    else:
        'В датасете нет фич времени. Запустите сначала функцию сreate_time_features'

    if tags_cols is None:
        tags_cols = [
            'month',
            'weekday',
            'is_weekend',
            'is_academic_year',
            'is_first_september',
            'is_week_before_first_september',
            'is_early_morning',
            'is_morning',
            'is_day',
            'is_evening',
            'is_late_evening',
            'is_night'
        ]

    if tags_dict is None:
        tags_dict = {
            'month': {7: 'июль', 8: 'август', 9: 'сентябрь'},
            'weekday': {
                0: 'понедельник',
                1: 'вторник',
                2: 'среда',
                3: 'четверг',
                4: 'пятница',
                5: 'суббота',
                6: 'воскресенье'
                },
            'is_weekend': {0:'рабочий_день', 1: 'выходной'},
            'is_academic_year': {0: 'каникулы', 1: 'учебный_год'},
            'is_first_september': {0: '', 1: 'первое_сентября'},
            'is_week_before_first_september': {0: '', 1: 'неделя_перед_учебным_годом'},
            'is_early_morning': {0: '', 1: 'раннее_утро'},
            'is_morning': {0: '', 1: 'утро'},
            'is_day': {0: '', 1: 'день'},
            'is_evening': {0: '', 1: 'вечер'},
            'is_late_evening': {0: '', 1: 'поздний_вечер'},
            'is_night': {0: '', 1: 'ночь'}
        }

    tags_cols_prefix = [f'{prefix}{col}' for col in tags_cols]

    tags_df = df[tags_cols_prefix].copy()
    tags_df = tags_df.fillna('')

    for col, col_prefix in zip(tags_cols, tags_cols_prefix):
        tags_df[col_prefix] = tags_df[col_prefix].replace(tags_dict[col])

    tags_df['tags'] = ''

    for col in tags_cols_prefix:
        tags_df['tags'] += tags_df[col] + ' '

    tags_df['tags'] = tags_df['tags'].apply(
        lambda x: delimiter.join( [x_spl.strip() for x_spl in x.split()] )
        )

    return tags_df['tags'].str.strip()


In [89]:
df = pd.DataFrame(
    data=[
        ['2021-09-01 04:54:00', 'MSK'],
        ['2021-09-26 04:54:00', 'MSK'],
        ['2021-09-17 04:54:00', 'MSK'],
        ['2021-09-17 05:54:00', 'MSK'],
        ['2021-09-17 08:54:00', 'MSK'],
        ['2021-09-17 11:54:00', 'MSK'],
        ['2021-09-17 18:54:00', 'MSK'],
        ['2021-09-17 21:54:00', 'MSK'],
        ['2021-09-17 23:54:00', 'MSK'],
        ['2021-09-17 0:54:00', 'MSK'],
        ['2021-09-17 15:54:00', 'MSK'],
        ['2021-07-04 18:07:40', 'MSK'],
        ['2021-07-04 18:07:40', 'MSK+6'],
        ['2021-07-05 18:07:40', 'MSK+6'],
        ['1970-01-01 00:00:00', 'MSK+6'],
        ['2021-08-04 13:34:29', 'MSK'],
        ['2021-08-04 13:34:29', 'MSK-1'],
        ['2021-08-04 13:34:29', np.nan]
        ],
    columns=['created', 'shift']
    )
display(df)

,created,shift
0,2021-09-01 04:54:00,MSK
1,2021-09-26 04:54:00,MSK
2,2021-09-17 04:54:00,MSK
3,2021-09-17 05:54:00,MSK
4,2021-09-17 08:54:00,MSK
5,2021-09-17 11:54:00,MSK
6,2021-09-17 18:54:00,MSK
7,2021-09-17 21:54:00,MSK
8,2021-09-17 23:54:00,MSK
9,2021-09-17 0:54:00,MSK


In [101]:
df_time_features_loc = create_time_features(df, dt_target='loc', fill_shift_na=False)
get_tags_from_time_features(df_time_features_loc, tags_cols=None, tags_dict=None, delimiter=';')
tags = df.join(df_time_features_loc).join(
    get_tags_from_time_features(df_time_features_loc, tags_cols=None, tags_dict=None, delimiter=';')
)
display(tags[['tags']])
print(tags['tags'][0])
print(tags['tags'][14])

,tags
0,сентябрь;среда;рабочий_день;учебный_год;первое...
1,сентябрь;воскресенье;выходной;учебный_год;ночь
2,сентябрь;пятница;рабочий_день;учебный_год;ночь
3,сентябрь;пятница;рабочий_день;учебный_год;ранн...
4,сентябрь;пятница;рабочий_день;учебный_год;утро
5,сентябрь;пятница;рабочий_день;учебный_год;день
6,сентябрь;пятница;рабочий_день;учебный_год;вечер
7,сентябрь;пятница;рабочий_день;учебный_год;позд...
8,сентябрь;пятница;рабочий_день;учебный_год;позд...
9,сентябрь;пятница;рабочий_день;учебный_год;ночь


сентябрь;среда;рабочий_день;учебный_год;первое_сентября;ночь



In [49]:
df.join(create_time_features(df, dt_target='msk', fill_shift_na=False))

,created,shift,msk_minute,msk_hour,msk_weekday,msk_day,msk_month,msk_is_academic_year,msk_is_weekend
0,2021-09-17 15:54:00,MSK,54.0,15.0,4.0,17.0,9.0,1.0,0.0
1,2021-07-04 18:07:40,MSK,7.0,18.0,6.0,4.0,7.0,0.0,1.0
2,2021-07-04 18:07:40,MSK+6,7.0,18.0,6.0,4.0,7.0,0.0,1.0
3,2021-07-05 18:07:40,MSK+6,7.0,18.0,0.0,5.0,7.0,0.0,0.0
4,1970-01-01 00:00:00,MSK+6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-08-04 13:34:29,MSK,34.0,13.0,2.0,4.0,8.0,0.0,0.0
6,2021-08-04 13:34:29,MSK-1,34.0,13.0,2.0,4.0,8.0,0.0,0.0
7,2021-08-04 13:34:29,NaN,34.0,13.0,2.0,4.0,8.0,0.0,0.0


# Создание фичей для трейна и теста

In [22]:
test_dt_loc_features = create_time_features(test, dt_target='loc')

In [23]:
train_dt_loc_features = create_time_features(train, dt_target='loc')

In [28]:
display(train_dt_loc_features['loc_year'].value_counts())
display(test_dt_loc_features['loc_year'].value_counts())

2021    44854283
1970         233
Name: loc_year, dtype: int64

2021    11213560
1970          69
Name: loc_year, dtype: int64

In [29]:
display(train_dt_loc_features['loc_month'].value_counts())
display(test_dt_loc_features['loc_month'].value_counts())

8    19696632
9    14856099
7    10301552
1         233
Name: loc_month, dtype: int64

8    4921414
9    3715914
7    2576232
1         69
Name: loc_month, dtype: int64

In [31]:
test_dt_loc_features[(test_dt_loc_features['loc_month']==1) & (test_dt_loc_features['loc_year']!=1970)]

,loc_minute,loc_hour,loc_weekday,loc_day,loc_month,loc_year,loc_is_weekend


In [32]:
train_dt_loc_features[(train_dt_loc_features['loc_month']==1) & (train_dt_loc_features['loc_year']!=1970)]

,loc_minute,loc_hour,loc_weekday,loc_day,loc_month,loc_year,loc_is_weekend
